# Pytorch Neural Networks Tutorial

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import numpy as np
import pandas as pd

# Load Data

In [ ]:
train = pd.read_csv("../data/digit-recognizer/train.csv")
test = pd.read_csv("../data/digit-recognizer/test.csv")

train.head()

In [ ]:
X_train = train.drop(columns="label").values.reshape(-1, 1, 28, 28) / 255
y_train = train["label"].values

In [ ]:
X_test = test.values.reshape(-1, 1, 28, 28) / 255

In [ ]:
X_train.shape

In [ ]:
X_test.shape

# Define NN

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

# Training

In [ ]:
inputs = torch.from_numpy(X_train).type(torch.FloatTensor)
labels = torch.from_numpy(y_train)

In [ ]:
print(inputs.size())
print(labels.size())

In [ ]:
net = Net()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optimizer = optim.Adam(net.parameters())
EPOCHS = 50

In [ ]:
log_dir = f"../src/runs/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
print(f"Tensorboard log directory : {log_dir}")
writer = SummaryWriter(log_dir=log_dir)

for epoch in range(EPOCHS):  # loop over the dataset multiple times
    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()

    optimizer.step()

    epoch_loss = loss.item()
    writer.add_scalar('loss', epoch_loss, epoch)
    print("epoch : ", epoch + 1, "loss : ", epoch_loss)

# Save

In [ ]:
PATH = './digit.pth'
torch.save(net.state_dict(), PATH)

# Load

In [ ]:
PATH = '../src/digit.pth'
net = Net()
net.load_state_dict(torch.load(PATH))

In [ ]:
idx = np.random.randint(X_test.shape[0])
print(idx)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(X_test[idx, 0, :])
plt.show()

In [ ]:
with torch.no_grad():
    inputs = torch.from_numpy(X_test).type(torch.FloatTensor)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    predicted_np = predicted.numpy()